In [1]:
%%time
import torch
import numpy as np
from scipy.optimize import minimize, Bounds
import cProfile, io, pstats
from pstats import SortKey
from botorch.models.gp_regression import SingleTaskGP
from botorch.acquisition.logei import qLogNoisyExpectedImprovement
from gpytorch import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll

torch.manual_seed(0)
np.random.seed(0)

torch.set_default_dtype(torch.double)

train_X = torch.rand(50, 2)
train_Y = torch.sin(train_X * 10).sum(dim=-1, keepdim=True)

model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
model.posterior(train_X).mean  # set caches
acqf = qLogNoisyExpectedImprovement(model, train_X, cache_root=False)

bounds = Bounds(
    lb=np.zeros(128),
    ub=np.ones(128),
    keep_feasible=True,
)
x0 = np.random.random(128)
None

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode
CPU times: user 2.14 s, sys: 917 ms, total: 3.06 s
Wall time: 5.79 s


In [2]:
%%time
def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = acqf(X_tensor).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxfun": 20},
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None

/opt/anaconda3/envs/botorch/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


         70166 function calls (67095 primitive calls) in 0.392 seconds

   Ordered by: cumulative time
   List reduced from 672 to 67 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.392    0.392 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.000    0.000    0.392    0.392 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
       46    0.001    0.000    0.389    0.008 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
       24    0.000    0.000    0.389    0.016 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
       23    0.000    0.000    0.389    0.017 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:16(wr

In [3]:
%%time
X = np.random.random(128)

with cProfile.Profile() as pr:
    for _ in range(20):
        f_and_grad(X)

CPU times: user 910 ms, sys: 775 ms, total: 1.69 s
Wall time: 294 ms
